In [1]:
# import dependencies

from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from transformers import AutoTokenizer, AutoModel


In [2]:
# connect to ES, create indexes
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

# to delete created indexes later on use:
# curl -XDELETE localhost:9200/label
# curl -XDELETE localhost:9200/document


04/30/2021 13:05:16 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:3.667s]
04/30/2021 13:05:16 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.004s]
04/30/2021 13:05:16 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.253s]
04/30/2021 13:05:16 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.003s]


In [3]:
# add data to ES

from lxml import etree
from lxml.etree import tostring

input_file_path = "data/Wikipedia-Strength-Training.xml"


tree = etree.parse(input_file_path)
root = tree.getroot()
# iterate through all the titles

document_dictionaries = []
for text_node in root.findall(".//text", namespaces=root.nsmap)[:20]:
    document_dictionaries.append({'text': text_node.text, 'meta': None})

document_store.write_documents(document_dictionaries)    


04/30/2021 13:06:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.686s]


In [7]:
# initialize retriever
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)


In [11]:
# initialize reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
mini_lm_reader = FARMReader("deepset/minilm-uncased-squad2", use_gpu=True)

04/30/2021 13:27:04 - INFO - farm.utils -   Using device: CPU 
04/30/2021 13:27:04 - INFO - farm.utils -   Number of GPUs: 0
04/30/2021 13:27:04 - INFO - farm.utils -   Distributed Training: False
04/30/2021 13:27:04 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
04/30/2021 13:27:13 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
04/30/2021 13:27:13 - INFO - farm.utils -   Using device: CPU 
04/30/2021 13:27:13 - INFO - farm.utils -   Number of GPUs: 0
04/30/2021 13:27:13 - INFO - farm.utils -   Distributed Training: False
04/30/2021 13:27:13 - INFO - farm.utils -   Automatic Mixed Precision: None
04/30/2021 13

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

04/30/2021 13:27:14 - INFO - filelock -   Lock 3050333871120 released on C:\Users\grizl/.cache\huggingface\transformers\fb46db721e5a48679af324906c6b2ac942ae359fd6c7c41940f704678c559b28.26f5a28983ca5d4829732c6f04277c607e503b7ca22e31bbf7113db821212965.lock
04/30/2021 13:27:15 - INFO - filelock -   Lock 3050348127280 acquired on C:\Users\grizl/.cache\huggingface\transformers\9e64fefefaba403b1a8230b21410a483b851458d65f2adf8ab8955084c44a7d5.9528c998736d06b774124cb23b7e6ebd40fc47e06576bfa588b9c049f58462be.lock


Downloading:   0%|          | 0.00/133M [00:00<?, ?B/s]

04/30/2021 13:27:20 - INFO - filelock -   Lock 3050348127280 released on C:\Users\grizl/.cache\huggingface\transformers\9e64fefefaba403b1a8230b21410a483b851458d65f2adf8ab8955084c44a7d5.9528c998736d06b774124cb23b7e6ebd40fc47e06576bfa588b9c049f58462be.lock
04/30/2021 13:27:23 - INFO - filelock -   Lock 3050317805216 acquired on C:\Users\grizl/.cache\huggingface\transformers\d6f79d7ce4a278c8df601e9b03444882b4ff0361599d7ee45976386bf254c113.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


...

04/30/2021 13:27:24 - INFO - filelock -   Lock 3050317805216 released on C:\Users\grizl/.cache\huggingface\transformers\d6f79d7ce4a278c8df601e9b03444882b4ff0361599d7ee45976386bf254c113.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
04/30/2021 13:27:26 - INFO - filelock -   Lock 3051513035552 acquired on C:\Users\grizl/.cache\huggingface\transformers\b9e16424436457ec053ecf35ac0073ecd3b3a558d17198f7a0dda663e42c665b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

04/30/2021 13:27:26 - INFO - filelock -   Lock 3051513035552 released on C:\Users\grizl/.cache\huggingface\transformers\b9e16424436457ec053ecf35ac0073ecd3b3a558d17198f7a0dda663e42c665b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
04/30/2021 13:27:26 - INFO - filelock -   Lock 3050338187152 acquired on C:\Users\grizl/.cache\huggingface\transformers\0b5762f31efa86856f88c656cc0b2dc9f9480e1031ac9b5f38a376a73501779c.6bf2f879cc715e2246d87b2dc885f23d13f8bf9a16a344bad68fd9d22b2378ce.lock


Downloading:   0%|          | 0.00/107 [00:00<?, ?B/s]

04/30/2021 13:27:27 - INFO - filelock -   Lock 3050338187152 released on C:\Users\grizl/.cache\huggingface\transformers\0b5762f31efa86856f88c656cc0b2dc9f9480e1031ac9b5f38a376a73501779c.6bf2f879cc715e2246d87b2dc885f23d13f8bf9a16a344bad68fd9d22b2378ce.lock
04/30/2021 13:27:27 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
04/30/2021 13:27:27 - INFO - farm.utils -   Using device: CPU 
04/30/2021 13:27:27 - INFO - farm.utils -   Number of GPUs: 0
04/30/2021 13:27:27 - INFO - farm.utils -   Distributed Training: False
04/30/2021 13:27:27 - INFO - farm.utils -   Automatic Mixed Precision: None
04/30/2021 13:27:27 - INFO - farm.infer -   Got ya 11 parallel workers to do inference ...
04/30/2021 13:27:27 - INFO - farm.infer -    0    0    0    0    0    0    0    0    0    0    0 
04/30/2021 13:27:27 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\  /|\  /w\  /w\  /w\
04/30/2021 13:27:27 - INFO - farm.infer -   /'\  /

In [12]:
# create pipeline

from haystack.pipeline import ExtractiveQAPipeline
# pipe = ExtractiveQAPipeline(reader, retriever)
pipe = ExtractiveQAPipeline(mini_lm_reader, retriever)

In [13]:
# make predictions
prediction1 = pipe.run(query="What is stamina?", top_k_retriever=10, top_k_reader=5)

prediction2 = pipe.run(query="How to avoid injury?", top_k_retriever=10, top_k_reader=5)


04/30/2021 13:27:35 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.019s]
Inferencing Samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.31s/ Batches]
04/30/2021 13:27:54 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.023s]
Inferencing Samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.00s/ Batches]


In [14]:
# get answers
print("Question 'What is stamina?'")
print_answers(prediction1, details="minimal")
print("Question 'How to avoid injury?'")
print_answers(prediction2, details="minimal")



Question 'What is stamina?'
[   {   'answer': 'cardiovascular',
        'context': 'e=2017-10-08 }}</ref> Although a greater endurance can '
                   'assist the [[cardiovascular]] system it does not imply '
                   'that any cardiovascular disease can be g'},
    {   'answer': 'cardiovascular',
        'context': 'e=2017-10-08 }}</ref> Although a greater endurance can '
                   'assist the [[cardiovascular]] system it does not imply '
                   'that any cardiovascular disease can be g'},
    {   'answer': 'cardiovascular',
        'context': 'e=2017-10-08 }}</ref> Although a greater endurance can '
                   'assist the [[cardiovascular]] system it does not imply '
                   'that any cardiovascular disease can be g'},
    {   'answer': 'cardiovascular',
        'context': 'e=2017-10-08 }}</ref> Although a greater endurance can '
                   'assist the [[cardiovascular]] system it does not imply '
                   '